In [213]:
import pandas as pd
import sqlite3
from datetime import datetime
import requests
import time
import asyncio
from telegram import Bot
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine
from tabulate import tabulate
import sys
import numpy as np

In [214]:
topic_id = 1

In [215]:
caminho = os.path.join(r"C:\Users\55359\Documents\My Projects\pipeline-dados\Dados", "dados_casa_moveis_decoracao.jsonl")

In [216]:
df = pd.read_json(caminho, lines=True, dtype={"preco_anterior": str, "fracao_preco_anterior":str, "preco_atual":str, "fracao_preco_atual":str})

In [204]:
def limpar_numero(valor):
            if pd.isnull(valor):
                return valor
            return int(str(valor).replace('.', '').replace(',', ''))

In [205]:
colunas_substituir_none = ['preco_anterior', 'fracao_preco_anterior', 'preco_atual', 'fracao_preco_atual']
df[colunas_substituir_none] = df[colunas_substituir_none].replace(['None', None], 0)

In [206]:
df["preco_anterior"] = df["preco_anterior"].apply(limpar_numero)
df["fracao_preco_anterior"] = df["fracao_preco_anterior"].apply(limpar_numero)
df["preco_atual"] = df["preco_atual"].apply(limpar_numero)
df["fracao_preco_atual"] = df["fracao_preco_atual"].apply(limpar_numero)

df['preco_anterior'] = df["preco_anterior"].astype(str) + "," + df["fracao_preco_anterior"].astype(str)
df['preco_atual'] = df["preco_atual"].astype(str) + "," + df["fracao_preco_atual"].astype(str)

df = df.drop(columns=['fracao_preco_anterior', 'fracao_preco_atual'])

df['porcentagem_desconto'] = df["porcentagem_desconto"].str.extract('(\d+)')
df['porcentagem_desconto'] = df['porcentagem_desconto'].fillna(0)
df['porcentagem_desconto'] = df['porcentagem_desconto'].astype(int)

df = df.sort_values('porcentagem_desconto', ascending=True)
df = df.drop_duplicates(subset=['titulo'], keep='last')
df['data_coleta']  = datetime.now()
df['topico_de_envio'] = f"{topic_id}"

In [207]:
df_modificado = df

In [208]:
df_modificado['preco_anterior'] = df_modificado['preco_anterior'].str.replace(',', '.').astype(float)
df_modificado['preco_atual'] = df_modificado['preco_atual'].str.replace(',', '.').astype(float)
df_modificado['desconto_reais'] = df_modificado['preco_anterior'] - df_modificado['preco_atual']

In [209]:
def calcular_pontuacao(row, lojas_vistas):
    pontuacao = 0
    
    if row["porcentagem_desconto"] >= 50:
        pontuacao += 5
    elif row["porcentagem_desconto"] >= 45:
        pontuacao += 4
    elif row["porcentagem_desconto"] >= 40:
        pontuacao += 2
    elif row["porcentagem_desconto"] >= 35:
        pontuacao += 1
    
    if row["desconto_reais"] > 1000 and row["porcentagem_desconto"] < 50:
        pontuacao += 5
    elif row["desconto_reais"] > 600 and row["porcentagem_desconto"] < 40:
        pontuacao += 4
    elif row["desconto_reais"] > 300 and row["porcentagem_desconto"] < 20:
        pontuacao += 3
    elif row["desconto_reais"] > 100 and row["porcentagem_desconto"] < 10:
        pontuacao += 3
    
    if pd.notna(row["highlight"]):
        pontuacao += 1
    if pd.notna(row["vendido_por"]):
        pontuacao += 1
    if pd.notna(row["detalhe_envio"]) or pd.notna(row["detalhe_envio_2"]):
        pontuacao += 1
    
    if pd.notna(row["vendido_por"]) and row["vendido_por"] in lojas_vistas and row["vendido_por"] != "Por Mercado Livre":
        pontuacao -= 3
    
    if pd.notna(row["vendido_por"]):
        lojas_vistas.add(row["vendido_por"])

    if "Kit" in str(row["titulo"]):
        pontuacao += 2

    if "Jogo De" in str(row["titulo"]):
        pontuacao += 2

    return pontuacao

lojas_vistas = set()

df["pontuacao"] = df.apply(calcular_pontuacao, axis=1, lojas_vistas=lojas_vistas)

def classificar_relevancia(pontuacao):
    if pontuacao >= 8:
        return "Relevante"
    elif pontuacao >= 6:
        return "Média-Relevância"
    elif pontuacao >= 2:
        return "Baixa-Relevância"
    else:
        return "Irrelevante"

df["relevancia"] = df["pontuacao"].apply(classificar_relevancia)

In [210]:
df_modificado['relevancia'].value_counts()

relevancia
Baixa-Relevância    585
Irrelevante         315
Média-Relevância     37
Relevante            11
Name: count, dtype: int64

In [211]:
df_modificado.sort_values('porcentagem_desconto', ascending=False).head()

,highlight,titulo,link,vendido_por,nota,total_avaliacoes,preco_anterior,preco_atual,porcentagem_desconto,detalhe_envio,detalhe_envio_2,imagem,data_coleta,topico_de_envio,desconto_reais,pontuacao,relevancia
63,MAIS VENDIDO,200.000 Bolinhas De Gel Para Pistolas - Cores ...,https://produto.mercadolivre.com.br/MLB-520081...,None,NaN,NaN,298.9,101.62,66,None,Chegará grátis segunda-feira,https://http2.mlstatic.com/D_Q_NP_2X_705293-ML...,2025-01-07 22:22:20.870794,1,197.28,7,Média-Relevância
810,None,Torneira Cozinha Gourmet Flexivel Misturador M...,https://www.mercadolivre.com.br/torneira-cozin...,None,NaN,NaN,599.6,239.60,60,Frete grátis,None,https://http2.mlstatic.com/D_Q_NP_2X_900291-ML...,2025-01-07 22:22:20.870794,1,360.00,6,Média-Relevância
797,None,Jogo De Lençol Casal Padrão 4 Peças 400 Fios E...,https://produto.mercadolivre.com.br/MLB-389256...,None,NaN,NaN,149.9,67.45,55,None,Chegará amanhã sábado,https://http2.mlstatic.com/D_Q_NP_2X_704998-ML...,2025-01-07 22:22:20.870794,1,82.45,8,Relevante
528,MAIS VENDIDO,Jogo De Colcha Bossa Mineral Queen Karsten,https://www.mercadolivre.com.br/jogo-de-colcha...,Por Mercado Livre,NaN,NaN,799.9,359.91,55,None,Chegará grátis amanhã sábado,https://http2.mlstatic.com/D_Q_NP_2X_728386-ML...,2025-01-07 22:22:20.870794,1,439.99,7,Média-Relevância
416,MAIS VENDIDO,Papel Higiênico Folha Dupla Neve Leve 16 Pague 15,https://produto.mercadolivre.com.br/MLB-337229...,Por Neve,NaN,NaN,48.9,23.60,51,None,None,https://http2.mlstatic.com/D_Q_NP_2X_709943-ML...,2025-01-07 22:22:20.870794,1,25.30,7,Média-Relevância


In [212]:
df_modificado.to_excel(r"C:\Users\55359\Downloads\base2.xlsx", index=False)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\55359\\Downloads\\base2.xlsx'